In [12]:
import pandas as pd
import numpy as np
import math
import copy
import QSTK.qstkutil.qsdateutil as du
import datetime as dt
import QSTK.qstkutil.DataAccess as da
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkstudy.EventProfiler as ep
import csv
import datetime as dt


def polio_val(raw_data):
    daily_data = copy.deepcopy(raw_data);
    polio=[1]
    dailyRet=[0]  
    for i in range(1,len(raw_data),1):
        fixedDay = (daily_data[i])/daily_data[0]
        polio.append(fixedDay)
        dailyRet.append((polio[i] - polio[i-1])/polio[i-1])
    
    print 'Sharpe Ratio of Fund : ' +  str(math.sqrt(252) * np.array(dailyRet).mean() / np.array(dailyRet).std())
    print 'Total Return of Fund : ' +  str(polio[len(polio) - 1])   
    print 'Standard Deviation of Fund :  ' + str(np.array(dailyRet).std())
    print 'Average Daily Return of Fund : ' + str(np.array(dailyRet).mean())



    
def price_matrix(dt_start, dt_end , ls_symbols):
    dt_timeofday = dt.timedelta(hours=16)
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
    c_dataobj = da.DataAccess('Yahoo')
    ls_keys = ['close']
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)   
    d_data = dict(zip(ls_keys, ldf_data))
    df_close = d_data['close']
    for s_key in ls_symbols:
        df_close[s_key] = df_close[s_key].fillna(method='ffill')
        df_close[s_key] = df_close[s_key].fillna(method='bfill')
    return df_close

def market_sim(filename='orders.csv' , cash=1000000):
    dates=[]
    symbols=[]
    reader = csv.reader(open(filename , 'rU') , delimiter=',')
    for row in reader:
        dates.append(dt.datetime(int(row[0]), int(row[1]), int(row[2])))
        symbols.append(row[3])
    
    #remove duplicate
    dates = list(set(dates))
    dates.sort()
    #多增加一天
    dates.append(dates[len(dates)-1] + dt.timedelta(days=1))
    
    
    symbols = list(set(symbols))
    dt_start = dates[0]
    dt_end = dates[len(dates)-1]
    dt_timeofday = dt.timedelta(hours=16)
    #dates为交易日期
    dates = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
  
    #构建交易矩阵
    trade_matrix = pd.DataFrame(0 , index=dates , columns=symbols)
    trade_matrix['CASH'] = cash
    
    reader = csv.reader(open(filename , 'rU') , delimiter=',')
    for row in reader:
        if(row[4] == 'Sell'):
            trade_matrix[row[3]].ix[dt.datetime(int(row[0]), int(row[1]), int(row[2])) + dt.timedelta(hours=16)] = -1 * int(row[5])
        else:
            trade_matrix[row[3]].ix[dt.datetime(int(row[0]), int(row[1]), int(row[2])) + dt.timedelta(hours=16)] = 1 * int(row[5])
   
    #仓位矩阵
    holds_matrix = copy.deepcopy(trade_matrix)
    
    
    for s_sym in symbols:
        for i in range(1, len(dates)):
            holds_matrix[s_sym].ix[dates[i]] = holds_matrix[s_sym].ix[dates[i-1]] + trade_matrix[s_sym].ix[dates[i]]   
    
    #求得前复权的收盘价格矩阵
    df_close = price_matrix(dt_start ,dt_end ,symbols)   
     
    for s_sym in symbols:
        holds_matrix['CASH'].ix[dates[0]] -= 1 * trade_matrix[s_sym].ix[dates[0]] * df_close[s_sym].ix[dates[0]]
  
    for i in range(1, len(dates)):
        last_cash = holds_matrix['CASH'].ix[dates[i-1]]
        trade_sum = 0
        for s_sym in symbols:
            trade_sum += 1 * trade_matrix[s_sym].ix[dates[i]] * df_close[s_sym].ix[dates[i]]
        holds_matrix['CASH'].ix[dates[i]] = last_cash - trade_sum       
    
    df_close['CASH'] = 1.0


    df_val =df_close * holds_matrix
    for s_sym in symbols:
        df_val['CASH']+=df_val[s_sym]
        
    
    return df_val['CASH'],dt_start ,dt_end 

    
    
if __name__ == '__main__':
    #模拟市场的每天的价值
    df_val,dt_start ,dt_end  = market_sim()
    df_val.to_csv('data/marketsim.csv')
    df_standard = price_matrix(dt_start ,dt_end ,['$SPX'])
    print 'Data Range :  '+str(dt_start)+' to  '+str(dt_end)
    #计算此组合的各项指标
    polio_val(df_val)
    #计算对比组合的各项指标
    print '\n'
    polio_val(df_standard['$SPX'])



[datetime.datetime(2011, 1, 10, 0, 0), datetime.datetime(2011, 1, 13, 0, 0), datetime.datetime(2011, 1, 26, 0, 0), datetime.datetime(2011, 2, 2, 0, 0), datetime.datetime(2011, 2, 10, 0, 0), datetime.datetime(2011, 3, 3, 0, 0), datetime.datetime(2011, 5, 3, 0, 0), datetime.datetime(2011, 6, 3, 0, 0), datetime.datetime(2011, 6, 10, 0, 0), datetime.datetime(2011, 8, 1, 0, 0), datetime.datetime(2011, 12, 20, 0, 0), datetime.datetime(2011, 12, 21, 0, 0)]
Data Range :  2011-01-10 00:00:00 to  2011-12-21 00:00:00
Sharpe Ratio of Fund : 1.23844571268
Total Return of Fund : 1.132562
Standard Deviation of Fund :  0.00696104058449
Average Daily Return of Fund : 0.000543063819063


Sharpe Ratio of Fund : 0.0183391412227
Total Return of Fund : 0.97759401457
Standard Deviation of Fund :  0.0149090969828
Average Daily Return of Fund : 1.72238432443e-05
